# 02 — Code Examples: Building Your First Agent

## What You'll Learn

This notebook teaches you how to build a working AI agent from scratch:

1. **Memory Integration**: Using the Memory class in practice
2. **Tool Creation**: Building callable functions agents can use
3. **Agent Loop**: Implementing decision-making and tool routing
4. **Integration**: Combining all components into a functioning system

## Why This Matters

**Real-World Context**:
- Agents need tools to interact with external systems (APIs, databases, search)
- Tool selection is core to agent intelligence (choosing the right tool for the task)
- Agent loops orchestrate the flow: input → reasoning → tool use → output

**What Makes an Agent**:
- **Perception**: Understanding user input
- **Decision**: Choosing which tool to use
- **Action**: Executing tools and returning results
- **Memory**: Maintaining context across interactions

## Prerequisites

- Completed [01_basics_overview.ipynb](01_basics_overview.ipynb)
- Understanding of Python functions and classes
- Basic knowledge of control flow (if/else, loops)

## Learning Path

This notebook is **self-contained** - all code runs without external dependencies. We'll build:
- A complete Memory implementation (copied from notebook 01)
- Two example tools (wiki search, SQL query)
- A minimal agent loop that routes requests to tools
- Integration examples showing it all working together

Let's start building!

## Step 1: Memory Foundation

### Why We Need Memory

**The Problem**:
- Agents need to remember previous interactions
- Context is essential for coherent conversations
- Without memory, every interaction starts from scratch

**Self-Contained Design**:
- This notebook includes its own Memory class (copied from notebook 01)
- Makes the notebook fully runnable without imports
- Good practice for production: keep dependencies clear

**What This Enables**:
- Stateful conversations
- Personalized responses
- Learning from user feedback
- Debugging conversation flow

In [ ]:
# Memory class (self-contained copy for this notebook)

class Memory:
    """
    Conversation memory with character-based limiting.
    
    This is a simplified copy from notebook 01 for self-contained execution.
    In production, you would import from a shared module.
    
    Key features:
    - Stores conversation history (role + content)
    - Limits total characters to stay within budgets
    - Provides summarization for system prompts
    - FIFO (First In, First Out) removal when full
    """
    
    def __init__(self, char_limit=500):
        """
        Initialize memory with character limit.
        
        Args:
            char_limit: Max characters to store (default 500)
        """
        self.char_limit = int(char_limit)
        self.history = []  # List of {"role": str, "content": str}
        self._total_chars = 0  # Running total for efficiency
    
    def add(self, role, content):
        """
        Add a message to memory, trimming oldest if needed.
        
        Args:
            role: 'user', 'assistant', or 'system'
            content: Message text
        
        Behavior:
        - Appends new message
        - Removes oldest messages until under char_limit
        - Maintains chronological order
        """
        item = {"role": role, "content": content}
        self.history.append(item)
        self._total_chars += len(content)
        
        # Trim from the beginning (oldest first)
        while self._total_chars > self.char_limit and self.history:
            removed = self.history.pop(0)
            self._total_chars -= len(removed['content'])
    
    def summarize(self, max_items=5):
        """
        Create compact summary of recent messages.
        
        Args:
            max_items: Number of recent messages to include
        
        Returns:
            String with messages joined by ' | '
        """
        # Extract content from last max_items messages
        return ' | '.join(m['content'] for m in self.history[-max_items:])
    
    def to_system_prompt(self):
        """
        Convert memory into system prompt format.
        
        Returns:
            Formatted string for LLM context injection
        """
        # Return empty if no history, otherwise format with prefix
        return 'Memory summary: ' + self.summarize() if self.history else ''

# Demo: Initialize and test memory
print("=== Memory Setup Demo ===\n")

# Create memory with 300 character limit
mem = Memory(char_limit=300)
print(f"Created Memory with {mem.char_limit} character limit")

# Add user preference
mem.add('user', 'I like concise summaries')
print("✓ Added user preference")

# Add assistant acknowledgment
mem.add('assistant', 'Will keep it brief')
print("✓ Added assistant acknowledgment\n")

# Check memory state
print(f"Messages stored: {len(mem.history)}")
print(f"Characters used: {mem._total_chars} / {mem.char_limit}\n")

# View summary
print('Memory summary:')
print(mem.summarize())
print("\n✓ Memory is ready for agent integration!")

Memory summary: I like concise summaries | Will keep it brief


### What Just Happened?

**Memory Initialization**:
- Created Memory instance with 300 character budget
- Added user preference and assistant response
- Tracked character usage automatically

**Key Behaviors**:
- Messages stored in chronological order
- Character count updated on each add
- Summary generation available for context injection
- Ready to integrate with agent tools

**Why Self-Contained**:
- No external imports needed - notebook runs standalone
- Easy to understand complete implementation
- Production pattern: import from shared modules
- Good for learning and experimentation

## Step 2: Creating Agent Tools

### What Are Tools?

**Definition**:
- Tools are callable functions that agents use to perform actions
- They extend agent capabilities beyond text generation
- Each tool has a specific purpose and structured output

**Real-World Tool Examples**:
- **Search**: Query databases, APIs, or knowledge bases
- **Compute**: Run calculations, data analysis, or simulations
- **Write**: Create files, send emails, update databases
- **Read**: Fetch data from external sources

### Tool Design Principles

**1. Single Responsibility**:
- Each tool does one thing well
- Clear, focused purpose
- Easy to test and maintain

**2. Structured Returns**:
- Return dictionaries or objects, not just strings
- Include metadata (source, timestamp, confidence)
- Enable agent to make decisions based on structure

**3. Error Handling**:
- Return error information in structured format
- Don't crash - return useful feedback
- Help agent recover and try alternatives

### Tools We'll Build

1. **search_wiki**: Simulates knowledge base search
2. **run_sql**: Simulates database query execution

These are "stubs" (fake implementations) to show the pattern - in production, they'd call real APIs.

In [ ]:
# Tool stub examples with structured returns

def search_wiki(query):
    """
    Simulate searching a knowledge base or wiki.
    
    Args:
        query: Search string (e.g., "LangGraph", "Python asyncio")
    
    Returns:
        Dictionary with structured information:
        - source: Where the data came from
        - query: Original search query
        - summary: Short description of results
        - url: Link to full information
    
    Real-world equivalent:
        - Call Wikipedia API
        - Query internal knowledge base
        - Search documentation with semantic search
    
    Why structured returns:
        - Agent can extract specific fields (summary, url)
        - Include metadata for decision-making
        - Easy to extend with more fields (confidence, timestamp)
        - Better than plain text for programmatic use
    """
    # Simulate search result with structured data
    return {
        "source": "wiki",
        "query": query,
        "summary": f"Summary for: {query}",
        "url": f"https://example.org/wiki/{query.replace(' ', '_')}"
    }

def run_sql(query):
    """
    Simulate running a SQL query against a database.
    
    Args:
        query: SQL query string (e.g., "SELECT * FROM users")
    
    Returns:
        Dictionary with query results:
        - rows: List of result rows (as dicts)
        - query: Original SQL query
    
    Real-world equivalent:
        - Connect to PostgreSQL, MySQL, SQLite
        - Use SQLAlchemy or raw DB drivers
        - Run query with parameterization
        - Handle connection pooling
    
    Why this pattern:
        - Return empty rows list if no results (not None)
        - Include original query for debugging
        - Rows as list of dicts for easy iteration
        - Could add: row_count, execution_time, columns
    """
    # Simulate query result with fake data
    # In production: execute real SQL and return actual rows
    return {
        "rows": [{"id": 1, "value": 42}],
        "query": query
    }

# Example usage demonstration
print("=== Tool Testing ===\n")

print("1. Testing search_wiki:")
wiki_result = search_wiki('LangGraph')
print(f"   Source: {wiki_result['source']}")
print(f"   Query: {wiki_result['query']}")
print(f"   Summary: {wiki_result['summary']}")
print(f"   URL: {wiki_result['url']}")
print()

print("2. Testing run_sql:")
sql_result = run_sql('SELECT * FROM metrics LIMIT 1')
print(f"   Query: {sql_result['query']}")
print(f"   Rows returned: {len(sql_result['rows'])}")
print(f"   First row: {sql_result['rows'][0]}")
print()

print("✓ Both tools working with structured returns!")

{'source': 'wiki', 'query': 'LangGraph', 'summary': 'Summary for: LangGraph', 'url': 'https://example.org/wiki/LangGraph'}
{'rows': [{'id': 1, 'value': 42}], 'query': 'SELECT 1'}


### What Just Happened?

**Tool Creation**:
- Built two function stubs simulating real tools
- Each returns structured dictionaries (not plain text)
- Includes metadata for agent decision-making

**Structured Returns Benefits**:
- **Parseable**: Agent can extract specific fields
- **Metadata**: Source, query, timestamps for context
- **Extensible**: Easy to add confidence scores, error codes
- **Type-Safe**: Predictable structure for agent logic

**From Stub to Production**:

**search_wiki** becomes:
```python
import requests
def search_wiki(query):
    response = requests.get(
        'https://en.wikipedia.org/api/rest_v1/page/summary/' + query
    )
    data = response.json()
    return {
        "source": "wikipedia",
        "query": query,
        "summary": data.get('extract', ''),
        "url": data.get('content_urls', {}).get('desktop', {}).get('page', '')
    }
```

**run_sql** becomes:
```python
import psycopg2
def run_sql(query):
    conn = psycopg2.connect(database="mydb")
    cursor = conn.execute(query)
    rows = [dict(row) for row in cursor.fetchall()]
    return {"rows": rows, "query": query}
```

## Step 3: Building the Agent Loop

### What Is an Agent Loop?

**Definition**:
An agent loop is the core decision-making cycle:
1. **Input**: Receive user request
2. **Reasoning**: Decide which tool to use (or if any)
3. **Action**: Execute the chosen tool
4. **Response**: Format results and return to user

**Production vs. This Demo**:

**In Production**:
- LLM decides which tool to use (function calling)
- Complex reasoning with multi-step plans
- Error handling and retry logic
- Chain multiple tools together

**In This Demo**:
- Simple keyword-based routing (if "wiki" → search_wiki)
- Single tool call per request
- Direct response generation
- Focus on understanding the pattern

### Agent Loop Components

**1. Input Processing**:
- Parse user request
- Extract intent and parameters
- Normalize text (lowercase, clean)

**2. Tool Selection**:
- Match request to available tools
- Choose best tool for the task
- Handle cases where no tool matches

**3. Tool Execution**:
- Call selected tool with parameters
- Capture structured results
- Handle errors gracefully

**4. Response Generation**:
- Format tool output for user
- Add context from memory
- Return both text response and raw data

### Why This Pattern Matters

- **Foundation for complex agents**: Same pattern scales to GPT-4 function calling
- **Debuggable**: Easy to trace decisions and outputs
- **Extensible**: Add tools without changing core logic
- **Testable**: Each component can be tested independently

In [ ]:
import re

def minimal_agent(user_input, memory=None):
    """
    A minimal agent loop demonstrating tool routing and execution.
    
    Args:
        user_input: User's request as text
        memory: Optional Memory instance for context tracking
    
    Returns:
        Tuple of (text_response, tool_output_dict)
    
    How it works:
    1. Parse input to detect intent (keywords, patterns)
    2. Route to appropriate tool based on intent
    3. Execute tool and capture results
    4. Format response and update memory
    5. Return both human-readable text and structured data
    
    Routing Logic:
    - Contains "wiki" OR matches "search wiki" → search_wiki
    - Contains "sql" OR matches SQL keywords → run_sql
    - No match → return fallback message
    
    Production evolution:
    - Replace keyword matching with LLM function calling
    - Add confidence scoring for tool selection
    - Support chaining multiple tools
    - Add error handling and retry logic
    """
    
    # Normalize input for easier matching
    user_lower = user_input.lower()
    
    # Tool routing: detect which tool to use
    
    # Route 1: Wiki search
    # Matches: "search wiki for X", "wiki X", "look up X in wiki"
    if 'wiki' in user_lower or re.search(r'search\s+wiki', user_lower):
        # Execute search_wiki tool
        out = search_wiki(user_input)
        
        # Format human-readable response
        text = f"I searched the wiki and found: {out['summary']} (source: {out['url']})"
        
        # Update memory if available
        if memory is not None:
            memory.add('user', user_input)
            memory.add('assistant', text)
        
        # Return both text and structured data
        return text, out
    
    # Route 2: SQL query
    # Matches: "run sql", "execute query", contains SQL keywords
    if 'sql' in user_lower or re.search(r'select|from|where', user_lower):
        # Execute run_sql tool
        out = run_sql(user_input)
        
        # Format response with row count
        text = f"SQL ran and returned {len(out['rows'])} rows."
        
        # Update memory if available
        if memory is not None:
            memory.add('user', user_input)
            memory.add('assistant', text)
        
        return text, out
    
    # Route 3: No tool matches (fallback)
    # Help user understand available capabilities
    fallback = 'I can search wiki or run sql. Try adding the word wiki or a SQL statement.'
    
    # Still update memory for conversation tracking
    if memory is not None:
        memory.add('user', user_input)
        memory.add('assistant', fallback)
    
    return fallback, None

# Demo: Run the agent with different inputs
print("=== Agent Loop Demo ===\n")

# Create fresh memory for this demo
mem = Memory(char_limit=300)
print("Created Memory for conversation tracking\n")

# Test 1: Wiki search
print("Test 1: Wiki Search")
print("User: 'Search wiki for LangGraph'")
response, tool_output = minimal_agent('Search wiki for LangGraph', memory=mem)
print(f"Agent: {response}")
print(f"Raw tool output: {tool_output}")
print()

# Test 2: SQL query
print("Test 2: SQL Query")
print("User: 'Run SQL: SELECT id, value FROM metrics'")
response, tool_output = minimal_agent('Run SQL: SELECT id, value FROM metrics', memory=mem)
print(f"Agent: {response}")
print(f"Raw tool output: {tool_output}")
print()

# Test 3: Unknown request (fallback)
print("Test 3: Unknown Request")
print("User: 'What's the weather?'")
response, tool_output = minimal_agent("What's the weather?", memory=mem)
print(f"Agent: {response}")
print(f"Raw tool output: {tool_output}")
print()

# Check memory after interactions
print("Memory after all interactions:")
print(mem.summarize())
print()
print(f"✓ Agent handled {len(mem.history)} interactions!")

('I searched the wiki and found: Summary for: Search wiki for LangGraph (source: https://example.org/wiki/Search_wiki_for_LangGraph)', {'source': 'wiki', 'query': 'Search wiki for LangGraph', 'summary': 'Summary for: Search wiki for LangGraph', 'url': 'https://example.org/wiki/Search_wiki_for_LangGraph'})
('SQL ran and returned 1 rows.', {'rows': [{'id': 1, 'value': 42}], 'query': 'Run SQL: SELECT id, value FROM metrics'})
Memory after interactions: I searched the wiki and found: Summary for: Search wiki for LangGraph (source: https://example.org/wiki/Search_wiki_for_LangGraph) | SQL ran and returned 1 rows.


### What Just Happened?

**Agent Loop Execution**:
1. Created Memory instance for conversation tracking
2. Processed three different user requests
3. Routed each to appropriate tool or fallback
4. Returned structured responses with metadata

**Routing Decisions Made**:
- "Search wiki" → Detected "wiki" keyword → Called search_wiki
- "Run SQL" → Detected SQL keywords → Called run_sql
- "Weather" → No match → Returned helpful fallback

**Key Observations**:
- **Memory Integration**: All interactions stored automatically
- **Structured Returns**: Both text and raw data available
- **Graceful Fallback**: Agent doesn't crash on unknown requests
- **Conversation Context**: Memory tracks full dialogue

**From Keywords to LLM Intelligence**:

**Current (keyword-based)**:
```python
if 'wiki' in user_lower:
    return search_wiki(user_input)
```

**Production (LLM function calling)**:
```python
# LLM decides which tool to use
tools = [search_wiki, run_sql]
tool_choice = llm.choose_tool(user_input, tools)
return tool_choice.execute(user_input)
```

**Benefits of LLM Routing**:
- Understands intent, not just keywords
- Handles variations ("look up X", "find info about X")
- Can chain multiple tools
- Learns from context and conversation history

**This Pattern Scales**:
- Add tools → just register in tool list
- Add LLM → replace routing logic
- Add multi-step → extend loop with plan/execute cycle
- Add monitoring → log decisions and outputs

## Practice Exercises

### Exercise 1: Add Confidence Scoring
**Objective**: Enhance tool selection with confidence scores.

**Requirements**:
- Add a confidence score (0.0-1.0) to tool selection logic
- Return confidence in the response text
- Lower confidence for ambiguous inputs
- High confidence for clear tool matches

**Example Implementation Start**:
```python
def minimal_agent_with_confidence(user_input, memory=None):
    user_lower = user_input.lower()
    confidence = 0.0
    
    # Exact match: high confidence
    if re.search(r'search\s+wiki', user_lower):
        confidence = 0.95
        # ... execute tool
    # Keyword present: medium confidence
    elif 'wiki' in user_lower:
        confidence = 0.70
        # ... execute tool
    
    return f"[Confidence: {confidence}] {response}", output
```

**Why This Matters**:
- Helps users understand agent certainty
- Enables threshold-based confirmations ("Are you sure?")
- Useful for debugging and monitoring
- Foundation for multi-agent voting systems

---

### Exercise 2: Pattern-Based Tool Registry
**Objective**: Replace hardcoded if/else with configurable tool registry.

**Requirements**:
- Create a list of (pattern, tool_function) pairs
- Iterate through registry to find matches
- Support multiple patterns per tool
- Add new tools without changing agent logic

**Example Implementation**:
```python
# Tool registry: list of (regex_pattern, tool_function, description)
TOOL_REGISTRY = [
    (r'wiki|search\s+wiki', search_wiki, "Search knowledge base"),
    (r'sql|select|from|where', run_sql, "Execute SQL query"),
]

def registry_agent(user_input, memory=None):
    user_lower = user_input.lower()
    
    # Iterate through registry to find match
    for pattern, tool, description in TOOL_REGISTRY:
        if re.search(pattern, user_lower):
            result = tool(user_input)
            # Format response...
            return response, result
    
    # No match found
    return fallback_message, None
```

**Benefits**:
- **Extensible**: Add tools by appending to list
- **Maintainable**: Tools self-document with descriptions
- **Testable**: Test pattern matching independently
- **Scalable**: Easy to add priority or confidence to registry

---

### Exercise 3: Multi-Tool Chaining (Advanced)
**Objective**: Enable agent to use multiple tools in sequence.

**Requirements**:
- Parse requests that need multiple tools
- Execute tools in correct order
- Pass output from one tool as input to next
- Return combined results

**Example Use Case**:
```
User: "Search wiki for Python, then query SQL for Python tutorials"

Agent plan:
1. search_wiki("Python") → get summary
2. run_sql("SELECT * FROM tutorials WHERE language='Python'")
3. Combine results into response
```

**Implementation Hints**:
- Detect compound requests (look for "then", "and", "after")
- Build execution plan as list of tool calls
- Execute sequentially, passing context between tools
- Combine outputs into coherent response

**Why This Matters**:
- Real agents often need multiple steps
- Foundation for planning and reasoning
- Enables complex workflows
- Core pattern in LangGraph and AutoGPT

---

### Exercise 4: Error Handling and Retry
**Objective**: Add robust error handling to tool execution.

**Requirements**:
- Wrap tool calls in try/except blocks
- Return structured error information
- Implement retry logic (max 3 attempts)
- Suggest alternatives on persistent failure

**Example Implementation**:
```python
def robust_agent(user_input, memory=None, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Select and execute tool
            tool = select_tool(user_input)
            result = tool(user_input)
            return format_success(result)
        except ToolExecutionError as e:
            if attempt == max_retries - 1:
                return format_error(e, suggest_alternatives=True)
            # Wait and retry
            time.sleep(2 ** attempt)  # Exponential backoff
```

**Production Patterns**:
- Log all errors for monitoring
- Track retry counts and success rates
- Implement circuit breakers for failing tools
- Provide meaningful error messages to users

---

## What You've Learned

### Core Concepts
1. **Memory Management**: Stateful conversations with character limiting
2. **Tool Creation**: Building callable functions with structured returns
3. **Agent Loop**: Decision-making cycle (input → route → execute → respond)
4. **Integration**: Combining all components into working system

### Production Path
- **Current**: Keyword-based routing with tool stubs
- **Next Step**: LLM-based tool selection (function calling)
- **Advanced**: Multi-step planning, error recovery, learning
- **Frameworks**: LangChain, LangGraph for production agents

### Key Takeaways
- Agents are orchestrators, not just LLM wrappers
- Tools extend capabilities beyond text generation
- Memory provides context and personalization
- Structured data enables programmatic decision-making

---

## Next Steps

Continue with [03_rag_kb.ipynb](03_rag_kb.ipynb) for:
- Retrieval-Augmented Generation (RAG) patterns
- Vector search and semantic retrieval
- Knowledge base integration
- Production-ready RAG implementation

**Your Agent Journey**:
1. [01_basics_overview.ipynb](01_basics_overview.ipynb): LLM fundamentals ✓
2. [02_code_examples.ipynb](02_code_examples.ipynb): Agent loops ✓
3. [03_rag_kb.ipynb](03_rag_kb.ipynb): RAG and knowledge bases ← Next
4. Course notebooks (L1-L4): Advanced frameworks

You're now ready to build real AI agents!